In [1]:
import pandas as pd
import tradingStrategy as ts
import datetime
import dateutil.relativedelta

In [2]:
stockDataFile = r"./StockData.xlsx"

In [3]:
# 样本内区间
sampleStartDate = datetime.datetime.strptime('2006-1-4', '%Y-%m-%d')
sampleEndDate = datetime.datetime.strptime('2013-12-31', '%Y-%m-%d')

# 样本外区间
backtestStartDate = datetime.datetime.strptime('2014-01-02', '%Y-%m-%d')
backtestEndDate = datetime.datetime.strptime('2021-12-31', '%Y-%m-%d')

initBalance = 1000000

shortMARange = range(1, 16)
mediumMARange = range(20, 101)

In [4]:
df = pd.read_excel(stockDataFile, sheet_name='399300', index_col='Date', parse_dates=['Date'])
df

,Open,High,Low,Close,Vol
Date,,,,,
2005-01-04,994.76,994.76,980.65,982.79,7412869
2005-01-05,981.57,997.32,979.87,992.56,7119109
2005-01-06,993.33,993.78,980.33,983.17,6288028
2005-01-07,983.04,995.71,979.81,983.95,7298694
2005-01-10,983.76,993.95,979.78,993.87,5791697
...,...,...,...,...,...
2021-12-27,4918.15,4939.69,4894.16,4919.32,124100952
2021-12-28,4923.69,4959.15,4913.60,4955.96,130457760
2021-12-29,4958.65,4958.65,4883.48,4883.48,113587000


In [5]:
stepMonth = 12  # 滑动窗口步长: 6个月
sampleDataSize = 8  # 样本内数据量: 9年

In [6]:
# 计算每个窗口
slidingWindowBacktestStartDateList = []  # List, 存每个窗口样本外开始日期
slidingWindowBacktestEndDateList = []  # List, 存每个窗口样本外结束日期

delta = backtestStartDate  # 循环控制条件
while (delta < backtestEndDate):  # 以相应步长遍历样本外区间

    # 计算窗口样本外开始日期
    startDate = df.loc[delta.strftime('%Y-%m')].index[0]  # 该窗口的样本外开始日期:这个月的第一个交易日
    slidingWindowBacktestStartDateList.append(startDate)  # 加入List中

    # 计算窗口样本外结束日期
    try:
        endDate = df.loc[(delta + dateutil.relativedelta.relativedelta(months=stepMonth - 1)).strftime('%Y-%m')].tail(
            1).index[0]  # 该窗口的样本外结束日期:(开始日期 + 步长 - 1)的月份最后一天
        slidingWindowBacktestEndDateList.append(endDate)  # 加入List中
    except Exception as e:
        # 抛出异常, 说明最后一个窗口不满步长, 该窗口样本内外结束日期为样本外结束日期
        slidingWindowBacktestEndDateList.append(backtestEndDate)

    delta = delta + dateutil.relativedelta.relativedelta(months=stepMonth)  # 循环控制条件增加相应步长

In [7]:
bestDf = pd.DataFrame(
    data={'样本内数据年份数': sampleDataSize, '步长/月': stepMonth,
          '样本外开始': slidingWindowBacktestStartDateList, '样本外结束': slidingWindowBacktestEndDateList},
    columns=['样本内数据年份数', '步长/月',
             '样本内开始', '样本内结束',
             '样本外开始', '样本外结束',
             '最佳短期均线', '最佳中期均线', '样本内净资产'])

bestDf['样本内开始'] = bestDf['样本外开始'].apply(
    lambda x: df.loc[(x - dateutil.relativedelta.relativedelta(years=sampleDataSize)).strftime('%Y-%m')].index[0]
)  # 样本内开始 = (样本外开始 - 步长)那个月的第一个交易日

bestDf['样本内结束'] = bestDf['样本外开始'].apply(
    lambda x: df.loc[(x - dateutil.relativedelta.relativedelta(months=1)).strftime('%Y-%m')].tail(1).index[
        0]
)
# 样本内结束 = 样本外开始那个月的上一个月的最后一个交易日

bestDf

,样本内数据年份数,步长/月,样本内开始,样本内结束,样本外开始,样本外结束,最佳短期均线,最佳中期均线,样本内净资产
0,8,12,2006-01-04,2013-12-31,2014-01-02,2014-12-31,NaN,NaN,NaN
1,8,12,2007-01-04,2014-12-31,2015-01-05,2015-12-31,NaN,NaN,NaN
2,8,12,2008-01-02,2015-12-31,2016-01-04,2016-12-30,NaN,NaN,NaN
3,8,12,2009-01-05,2016-12-30,2017-01-03,2017-12-29,NaN,NaN,NaN
4,8,12,2010-01-04,2017-12-29,2018-01-02,2018-12-28,NaN,NaN,NaN
5,8,12,2011-01-04,2018-12-28,2019-01-02,2019-12-31,NaN,NaN,NaN
6,8,12,2012-01-04,2019-12-31,2020-01-02,2020-12-31,NaN,NaN,NaN
7,8,12,2013-01-04,2020-12-31,2021-01-04,2021-12-31,NaN,NaN,NaN


In [8]:
# 将bestDf向量化
slidingWindowSampleStartDateList = bestDf['样本内开始'].values
slidingWindowSampleEndDateList = bestDf['样本内结束'].values
slidingWindowBacktestStartDateList = bestDf['样本外开始'].values
slidingWindowBacktestEndDateList = bestDf['样本外结束'].values

bestShortMAList = bestDf['最佳短期均线'].values
bestMediumMAList = bestDf['最佳中期均线'].values
bestNetAssetList = bestDf['样本内净资产'].values

In [9]:
# 使用最佳参数回测交易明细
backtestTradingDf = pd.DataFrame(columns=['Open', 'Close', 'sign', 'hold', 'balance', 'netAsset', 'profit'])

In [10]:
balance = initBalance  # 样本外区间进行回测时持有的现金
hold = 0  # 样本外区间进行回测时持有的证券数量

# 遍历bestDf, 计算每个窗口的最佳参数组合
for i in range(len(bestNetAssetList)):
    # 进度
    print(str(i + 1) + '/' + str(len(bestNetAssetList)))

    ############样本内区间############
    # 样本内区间, 计算最佳参数
    iSampleStartDate = slidingWindowSampleStartDateList[i]  # 该窗口样本内开始时间
    iSampleEndDate = slidingWindowSampleEndDateList[i]  # 该窗口样本内结束时间

    itAllStrategyDf = ts.getAllStrategy(df, iSampleStartDate, iSampleEndDate, shortMARange, mediumMARange)  # 计算该窗口所有参数
    bestShortMA, bestMediumMA, bestNetAsset = ts.getBestStrategy(itAllStrategyDf)  # 找最佳参数

    # 写入最佳参数
    bestShortMAList[i] = bestShortMA
    bestMediumMAList[i] = bestMediumMA
    bestNetAssetList[i] = bestNetAsset

    ############样本外区间############
    # 使用最佳参数在该窗口进行回测
    iBacktestStartDate = slidingWindowBacktestStartDateList[i]  # 该窗口样本外开始时间
    iBacktestEndDate = slidingWindowBacktestEndDateList[i]  # 该窗口样本外结束时间

    backtestStrategyDf = ts.strategy(df, iBacktestStartDate, iBacktestEndDate, bestShortMA, bestMediumMA,
                                     hold)  # 该窗口买卖标志位
    newBacktestTradingDf = ts.trading(backtestStrategyDf, balance, hold)  # 该窗口交易明细

    balance = newBacktestTradingDf.iloc[-1]['balance']  # 更新现金
    hold = newBacktestTradingDf.iloc[-1]['hold']  # 更新持有证券数量

    backtestTradingDf = pd.concat([backtestTradingDf, newBacktestTradingDf])  # 将该窗口交易明细合并进回测交易明细中

1/8
hortMA = 3, mediumMA = 31, netAsset = 4836044.545325025shortMA = 3, mediumMA = 91, netAsset = 4831495.667495013shortMA = 4, mediumMA = 71, netAsset = 5076444.511655013shortMA = 5, mediumMA = 52, netAsset = 5303760.275665015shortMA = 6, mediumMA = 32, netAsset = 5123364.577345012shortMA = 6, mediumMA = 94, netAsset = 4133493.837960008shortMA = 7, mediumMA = 76, netAsset = 5727880.51433501shortMA = 8, mediumMA = 57, netAsset = 5130192.024160013shortMA = 9, mediumMA = 39, netAsset = 5309547.586215012shortMA = 10, mediumMA = 21, netAsset = 4453247.424395017shortMA = 10, mediumMA = 83, netAsset = 5195447.88106501shortMA = 11, mediumMA = 64, netAsset = 5323204.1868550135shortMA = 12, mediumMA = 47, netAsset = 5158692.366800012shortMA = 13, mediumMA = 30, netAsset = 4331361.257145018shortMA = 13, mediumMA = 94, netAsset = 4453131.796895005shortMA = 14, mediumMA = 76, netAsset = 4496255.995960013shortMA = 15, mediumMA = 58, netAsset = 4257762.150905005
ortMA = 5, mediumMA = 23, netAsset = 

In [11]:
bestDf

,样本内数据年份数,步长/月,样本内开始,样本内结束,样本外开始,样本外结束,最佳短期均线,最佳中期均线,样本内净资产
0,8,12,2006-01-04,2013-12-31,2014-01-02,2014-12-31,7,41,7576352.11915
1,8,12,2007-01-04,2014-12-31,2015-01-05,2015-12-31,6,79,4887152.4309
2,8,12,2008-01-02,2015-12-31,2016-01-04,2016-12-30,6,39,2570290.728495
3,8,12,2009-01-05,2016-12-30,2017-01-03,2017-12-29,1,42,3110051.208335
4,8,12,2010-01-04,2017-12-29,2018-01-02,2018-12-28,6,30,2132626.852475
5,8,12,2011-01-04,2018-12-28,2019-01-02,2019-12-31,1,42,1758644.51684
6,8,12,2012-01-04,2019-12-31,2020-01-02,2020-12-31,10,25,2122122.60717
7,8,12,2013-01-04,2020-12-31,2021-01-04,2021-12-31,8,55,2401897.58212


In [12]:
backtestTradingDf

,Open,Close,sign,hold,balance,netAsset,profit
2014-01-02,2323.43,2321.98,0.0,0.0,1.000000e+06,1.000000e+06,0.00000
2014-02-18,2310.31,2282.44,1.0,432.0,1.447053e+03,9.995010e+05,0.00000
2014-02-27,2170.81,2154.11,-1.0,0.0,9.387681e+05,9.387681e+05,-61231.92192
2014-04-11,2269.56,2270.67,1.0,413.0,9.711339e+02,9.382994e+05,0.00000
2014-04-30,2158.76,2158.66,-1.0,0.0,8.920932e+05,8.920932e+05,-46674.84808
...,...,...,...,...,...,...,...
2021-07-09,5063.15,5069.44,-1.0,0.0,1.390553e+06,1.390553e+06,-52204.07879
2021-10-18,4922.51,4874.78,1.0,282.0,1.711080e+03,1.389859e+06,0.00000
2021-11-04,4841.09,4868.74,-1.0,0.0,1.366216e+06,1.366216e+06,-24337.10760
2021-12-09,5000.69,5078.69,1.0,273.0,3.449017e+02,1.365533e+06,0.00000


In [13]:
backtestNetAsset = backtestTradingDf.iloc[-1]['netAsset']
backtestTotalRate = ts.getTotalRate(initBalance, backtestNetAsset)
backtestCompoundRate = ts.getCompoundRate(backtestTotalRate, backtestStartDate, backtestEndDate)

In [19]:
revenues = pd.Series([sampleDataSize, stepMonth, backtestNetAsset, backtestCompoundRate],
                     index=['样本内数据年份数', '步长/月', '样本外净资产', '年均收益率复利'])

样本内数据年份数    8.000000e+00
步长/月        1.200000e+01
样本外净资产      1.349066e+06
年均收益率复利     3.812245e-02
dtype: float64

In [8]:
df

,Open,High,Low,Close,Vol
Date,,,,,
2005-01-04,994.76,994.76,980.65,982.79,7412869
2005-01-05,981.57,997.32,979.87,992.56,7119109
2005-01-06,993.33,993.78,980.33,983.17,6288028
2005-01-07,983.04,995.71,979.81,983.95,7298694
2005-01-10,983.76,993.95,979.78,993.87,5791697
...,...,...,...,...,...
2021-12-27,4918.15,4939.69,4894.16,4919.32,124100952
2021-12-28,4923.69,4959.15,4913.60,4955.96,130457760
2021-12-29,4958.65,4958.65,4883.48,4883.48,113587000


In [10]:
df.loc['2005-01-09']

KeyError: '2005-01-09'

In [17]:
date = datetime.datetime.strptime('2005-01-09', '%Y-%m-%d')
date

datetime.datetime(2005, 1, 9, 0, 0)